# Collected rook errors 2022-12-08


In [ ]:
import os
os.environ['ROOK_URL'] = 'http://rook.dkrz.de/wps'

from rooki import operators as ops

## Process error: Requested levels include some not found in the dataset: 100000.0 

 b4cd76ca-6d80-11ed-b031-fa163eaaf620 | 22396 | execute   | 1.0.0   | 2022-11-26 12:51:47.496774 | 2022-11-26 12:51:50.651248 | orchestrate | Process error: Requested levels include some not found in the dataset: 100000.0 

```
drwx------ 3 tomcat tomcat 114 Dec  5 15:06 pywps_process_7kwbnw67

workflow:
<![CDATA[{\\\"inputs\\\": {\\\"ta\\\": [\\\"c3s-cmip6.CMIP.NOAA-GFDL.GFDL-ESM4.historical.r1i1p1f1.Amon.ta.gr1.v20190726\\\"]}, \\\"steps\\\": {\\\"subset_ta_1\\\": {\\\"run\\\": \\\"subset\\\", \\\"in\\\": {\\\"collection\\\": \\\"inputs/ta\\\", \\\"time\\\": \\\"1950/1999\\\", \\\"time_components\\\": \\\"year:1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999|month:jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec\\\", \\\"level\\\": \\\"10000\\\", \\\"area\\\": \\\"-157,49.2,-117.3,67.6\\\"}}}, \\\"outputs\\\": {\\\"output\\\": \\\"subset_ta_1/output\\\"}, \\\"doc\\\": \\\"workflow\\\"}]]>

  File "/usr/local/anaconda/envs/rook/lib/python3.10/site-packages/clisops/ops/subset.py", line 105, in _calculate
    result = subset_bbox(ds, **kwargs)
  File "/usr/local/anaconda/envs/rook/lib/python3.10/site-packages/clisops/core/subset.py", line 270, in func_checker
    return func(*args, **kwargs)
  File "/usr/local/anaconda/envs/rook/lib/python3.10/site-packages/clisops/core/subset.py", line 1295, in subset_bbox
    da = subset_level_by_values(da, level_values)
  File "/usr/local/anaconda/envs/rook/lib/python3.10/site-packages/clisops/core/subset.py", line 308, in func_checker
    return func(*args, **kwargs)
TypeError: subset_level_by_values() got multiple values for argument 'level_values'
```

In [ ]:
subset = ops.Subset(
        ops.Input(
            'noaa', ['c3s-cmip6.CMIP.NOAA-GFDL.GFDL-ESM4.historical.r1i1p1f1.Amon.ta.gr1.v20190726']
        ),
        time="1950/1950",
        level='10000',        
)

subset = ops.Subset(
        subset,
        level='10000',        
)

subset = ops.Subset(
    subset,
    area='-157,49.2,-117.3,67.6'
)

wf = subset

In [ ]:
resp = wf.orchestrate()
resp.ok

In [ ]:
ds = resp.datasets()[0]
ds